In [1]:
var('q t')

def number_of_ones(a):
    k = 0
    for i in range(len(a)):
        if a[i]==1:
            k = k+1
    return k

def my_len(a):
    return 2*(len(a)-number_of_ones(a))+number_of_ones(a)

def series_t_q(w):
    series = 0
    for i in range(len(w)):
        series_t = 0
        for j in range(len(w[i])):
            series_t = series_t + t^my_len(w[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(a, tail, is_greater):
    if len(a) < len(tail)-1:
        return False
    b = a[len(a)-len(tail)+1: len(a)]
    if b != tail[1:]:
        return False
    if is_greater:
        if a == b:
            return True
        if a[len(a)-len(tail)] > tail[0]:
            return True
    else:
        if a == b:
            return False
        if a[len(a)-len(tail)] == tail[0]:
            return True
    return False

def subset_ends(w, tail, is_greater):
    return [[b for b in a if partition_ends(b, tail, is_greater)] for a in w]

def vector_sum(u, v):
    return dict((x, u.get(x, 0) + v.get(x, 0)) for x in set(u)|set(v))

def vector_simpl(u):
    v = {}
    S = set()
    for a in set(u):
        b = sorted(a)
        b.reverse()
        b = tuple(b)
        S.add(b)
        v = vector_sum(v, {b:u[a]})
    return {a:v[a] for a in S if v[a] != 0}

def univ_vector_simpl1(a):
    for i in range(len(a)-1):
        if a[i]<a[i+1]:
            return {a[0:i]+(a[i]+a[i+1],)+a[i+2:len(a)]:a[i+1]-a[i], a[0:i]+(a[i+1],)+(a[i],)+a[i+2:len(a)]:1}
    return {a:1}

def univ_vector_simpl(u):
    v={}
    for a in set(u):
        w = univ_vector_simpl1(a)
        v = vector_sum(v, {b:w[b]*u[a] for b in set(w)})
    if v == u:
        return v
    return univ_vector_simpl(v)

def several_virasoro(a, u):
    return univ_vector_simpl({a+b:u[b] for b in set(u)})

def my_order(a):
    b = to_polynomial(a, sum(a))
    b = [-t for t in b]
    b.reverse()
    return ((my_len(a), len(a)-number_of_ones(a), b)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    a = list(Partitions(n))
    a.sort(key = my_order, reverse = True)
    return a

def is_subpartition(a,b):
    if len(a)<len(b):
        return false
    for i in range(len(a)-len(b)+1):
        if a[i:i+len(b)] == b:
            return true
    return false

def partition_contains_any_of_list(a, w):
    for b in w:
        if is_subpartition(a, b):
            return True
    return False

def partition_contains_any_of_Qv2(a):
    if a == []:
        return false
    if partition_contains_any_of_list(a, [[2], [3, 3], [4, 3, 1], [4, 4, 1], [6, 5, 3, 1]]):
        return true
    if len(a)>2:
        for i in range(len(a)-2):
            if a[i]-a[i+2]<=2:
                return true
    if len(a)>3:
        for i in range(len(a)-3):
            if a[i]-a[i+3]<=3 or (a[i]-a[i+3]<=4 and a[i+1]-a[i+2]<=3):
                return true
    if len(a)>4:
        for i in range(len(a)-4):
            if a[i]-a[i+1]==1 and a[i+1]-a[i+2]==2 and a[i+2]-a[i+3]==2 and a[i+3]-a[i+4]==1:
                return true
    return false

def partition_contains_any_of_Qv3(a):
    if a == []:
        return false
    if partition_contains_any_of_list(a, [[2], [3, 3], [4, 3, 1], [4, 4, 1], [6, 5, 3, 1], [3, 1, 1], [5, 4, 1, 1], [1, 1, 1]]):
        return true
    if len(a)>2:
        for i in range(len(a)-2):
            if a[i]-a[i+2]<=2:
                return true
    if len(a)>3:
        for i in range(len(a)-3):
            if a[i]-a[i+3]<=3 or (a[i]-a[i+3]<=4 and a[i+1]-a[i+2]<=3):
                return true
    if len(a)>4:
        for i in range(len(a)-4):
            if a[i]-a[i+1]==1 and a[i+1]-a[i+2]==2 and a[i+2]-a[i+3]==2 and a[i+3]-a[i+4]==1:
                return true
    return false

def matrix_UVir_m2_m3(n):
    m2 = {(1,1):3, (2,):-4}
    m3 = {(1,1,1):4, (2,1):-12, (3,):3}
    a = ordered_partitions(n)
    b = ordered_partitions(n-2)
    c = ordered_partitions(n-3)
    b_m2 = [several_virasoro(tuple(b[i]), m2) for i in range(len(b))]
    c_m3 = [several_virasoro(tuple(c[i]), m3) for i in range(len(c))]
    A = matrix({(i,j): b_m2[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(b))}, sparse = False)
    B = matrix({(i,j): c_m3[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(c))}, sparse = False)
    A = A.stack(B)
    A = A.rref()
    return A

def to_polynomial(a,n):
    b = [0]*n
    b[n-1] = number_of_ones(a)
    for i in range(len(a)):
        if a[i]!= 1:
            b[a[i]-2] = b[a[i]-2]+1
    return b

def pivots_up_to(n):
    w = []
    for i in range(n+1):
        a = ordered_partitions(i)
        A = matrix_UVir_m2_m3(i)
        w.append([a[i] for i in A.pivots()])
    return w

def base_up_to(n):
    w = []
    pivs = pivots_up_to(n)
    for i in range(n+1):
        a = ordered_partitions(i)
        w.append([a[j] for j in range(len(a)) if a[j] not in pivs[i]])
    return w

def base_up_tov2(n):
    base = []
    for i in range(n+1):
        a = ordered_partitions(i)
        add = [a[j] for j in range(len(a)) if not partition_contains_any_of_Qv2(a[j])]
        base = base + [add]
    return base

def base_up_tov3(n):
    base = []
    for i in range(n+1):
        a = ordered_partitions(i)
        add = [a[j] for j in range(len(a)) if not partition_contains_any_of_Qv3(a[j])]
        base = base + [add]
    return base

In [2]:
ordered_partitions(15)

[[2, 2, 2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 2, 1, 1, 1],
 [2, 2, 2, 2, 2, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 2, 2, 2, 2],
 [3, 2, 2, 2, 2, 2, 1, 1],
 [3, 2, 2, 2, 2, 1, 1, 1, 1],
 [3, 2, 2, 2, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 2, 2, 2, 2, 1],
 [4, 2, 2, 2, 2, 2, 1],
 [3, 3, 2, 2, 2, 1, 1, 1],
 [4, 2, 2, 2, 2, 1, 1, 1],
 [3, 3, 2, 2, 1, 1, 1, 1, 1],
 [4, 2, 2, 2, 1, 1, 1, 1, 1],
 [3, 3, 2, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 3, 2, 2, 2],
 [4, 3, 2, 2, 2, 2],
 [5, 2, 2, 2, 2, 2],
 [3, 3, 3, 2, 2, 1, 1],
 [4, 3, 2, 2, 2, 1, 1],
 [5, 2, 2, 2, 2, 1,

In [3]:
base_up_tov2(15)==base_up_to(15)

True

In [4]:
base_up_tov3(15)==base_up_to(15)

True

In [5]:
base_up_to(15)

[[[]],
 [[1]],
 [[1, 1]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 1, 1], [5, 1], [6]],
 [[4, 3], [5, 1, 1], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 1, 1], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]],
 [[5, 4, 1], [6, 3, 1], [5, 5], [6, 4], [7, 3], [8, 1, 1], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [7, 3, 1],
  [6, 5],
  [7, 4],
  [8, 3],
  [9, 1, 1],
  [10, 1],
  [11]],
 [[5, 5, 1, 1],
  [6, 4, 1, 1],
  [6, 5, 1],
  [7, 4, 1],
  [8, 3, 1],
  [6, 6],
  [7, 5],
  [8, 4],
  [9, 3],
  [10, 1, 1],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 1, 1],
  [7, 4, 1, 1],
  [6, 6, 1],
  [7, 5, 1],
  [8, 4, 1],
  [9, 3, 1],
  [7, 6],
  [8, 5],
  [9, 4],
  [10, 3],
  [11, 1, 1],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [7, 4, 3],
  [6, 6, 1, 1],
  [7, 5, 1, 1],
  [8, 4, 1, 1],
  [7, 6, 1],
  [8, 5, 1],
  [9, 4, 1],
  [10, 3, 1],
  [7, 7],
  [8, 6],
  [9, 5],
  [10, 4],
  [11, 3],
  [12, 1, 1],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [8, 4, 3],
  [7, 6, 1, 1],
 

In [6]:
pivots_up_to(15)

[[],
 [],
 [[2]],
 [[2, 1], [1, 1, 1]],
 [[2, 2], [2, 1, 1], [1, 1, 1, 1]],
 [[2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1], [3, 2], [3, 1, 1]],
 [[2, 2, 2],
  [2, 2, 1, 1],
  [2, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [3, 2, 1],
  [3, 1, 1, 1],
  [3, 3],
  [4, 2]],
 [[2, 2, 2, 1],
  [2, 2, 1, 1, 1],
  [2, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2],
  [3, 2, 1, 1],
  [3, 1, 1, 1, 1],
  [3, 3, 1],
  [4, 2, 1],
  [4, 1, 1, 1],
  [5, 2]],
 [[2, 2, 2, 2],
  [2, 2, 2, 1, 1],
  [2, 2, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 1],
  [3, 2, 1, 1, 1],
  [3, 1, 1, 1, 1, 1],
  [3, 3, 2],
  [4, 2, 2],
  [3, 3, 1, 1],
  [4, 2, 1, 1],
  [4, 1, 1, 1, 1],
  [4, 3, 1],
  [5, 2, 1],
  [5, 1, 1, 1],
  [6, 2]],
 [[2, 2, 2, 2, 1],
  [2, 2, 2, 1, 1, 1],
  [2, 2, 1, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 2],
  [3, 2, 2, 1, 1],
  [3, 2, 1, 1, 1, 1],
  [3, 1, 1, 1, 1, 1, 1],
  [3, 3, 2, 1],
  [4, 2, 2, 1],
  [3, 3, 1, 1, 1],
  

In [7]:
series_t_q(base_up_to(20))

(3*t^8 + 9*t^7 + 19*t^6 + 7*t^5 + 9*t^4 + t^3 + t^2)*q^20 + (t^8 + 7*t^7 + 16*t^6 + 7*t^5 + 8*t^4 + t^3 + t^2)*q^19 + (5*t^7 + 14*t^6 + 6*t^5 + 8*t^4 + t^3 + t^2)*q^18 + (3*t^7 + 11*t^6 + 6*t^5 + 7*t^4 + t^3 + t^2)*q^17 + (2*t^7 + 9*t^6 + 5*t^5 + 7*t^4 + t^3 + t^2)*q^16 + (7*t^6 + 5*t^5 + 6*t^4 + t^3 + t^2)*q^15 + (5*t^6 + 4*t^5 + 6*t^4 + t^3 + t^2)*q^14 + (3*t^6 + 4*t^5 + 5*t^4 + t^3 + t^2)*q^13 + (2*t^6 + 3*t^5 + 5*t^4 + t^3 + t^2)*q^12 + (3*t^5 + 4*t^4 + t^3 + t^2)*q^11 + (2*t^5 + 4*t^4 + t^3 + t^2)*q^10 + (t^5 + 3*t^4 + t^3 + t^2)*q^9 + (3*t^4 + t^3 + t^2)*q^8 + (2*t^4 + t^3 + t^2)*q^7 + (t^4 + t^3 + t^2)*q^6 + (t^3 + t^2)*q^5 + (t^3 + t^2)*q^4 + q^3*t^2 + q^2*t^2 + q*t + 1